In [1]:
!pip install huggingface_hub

In [2]:
from huggingface_hub import interpreter_login
interpreter_login()  # This will prompt for your HF token


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



In [3]:
from huggingface_hub import HfApi, create_repo
import os
import shutil

def push_to_hub(checkpoint_path, repo_name, organization=None):
    """
    Push LDM checkpoints to Hugging Face Hub
    
    Args:
        checkpoint_path (str): Path to your ldm_checkpoints folder
        repo_name (str): Name for your Hugging Face repository
        organization (str, optional): Organization name if pushing to an organization
    """
    # Initialize Hugging Face API
    api = HfApi()
    
    # Create the repository
    full_repo_name = f"{organization}/{repo_name}" if organization else repo_name
    create_repo(full_repo_name, repo_type="model", exist_ok=True)
    
    # Create a temporary directory to organize files
    temp_dir = "temp_upload"
    os.makedirs(temp_dir, exist_ok=True)
    
    try:
        # Copy and organize files
        for epoch_folder in os.listdir(checkpoint_path):
            epoch_path = os.path.join(checkpoint_path, epoch_folder)
            if os.path.isdir(epoch_path):
                # Create epoch directory in temp
                epoch_temp_path = os.path.join(temp_dir, epoch_folder)
                os.makedirs(epoch_temp_path, exist_ok=True)
                
                # Copy files
                for item in os.listdir(epoch_path):
                    src = os.path.join(epoch_path, item)
                    dst = os.path.join(epoch_temp_path, item)
                    if os.path.isfile(src):
                        shutil.copy2(src, dst)
        
        # Upload all files
        api.upload_folder(
            folder_path=temp_dir,
            repo_id=full_repo_name,
            repo_type="model"
        )
        
        print(f"Successfully pushed checkpoints to: https://huggingface.co/{full_repo_name}")
        
    finally:
        # Cleanup
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

# Example usage
checkpoint_path = "./ldm_checkpoints"  # Your checkpoints folder path
repo_name = "Latent-Diffusion-Model"  # Choose your repository name

# Push to your personal account
push_to_hub(checkpoint_path, repo_name,organization="vish26")


training_state.pth:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

training_state.pth:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

Successfully pushed checkpoints to: https://huggingface.co/vish26/Latent-Diffusion-Model
